In [6]:
%pip install groq
%pip install -q -U google-genai

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 188, in handle_pip_version_check
    pi

In [11]:
import os
import json
import time
import requests
from google import genai

class AIAgentSystem:
    def __init__(self, gemini_api_key=None, groq_api_key=None):
        # Initialize API keys
        self.gemini_api_key = gemini_api_key or os.environ.get("GEMINI_API_KEY")
        self.groq_api_key = groq_api_key or os.environ.get("GROQ_API_KEY")
        
        # Set up Gemini
        self.gemini_client = genai.Client(api_key=self.gemini_api_key)
        self.gemini_model = "gemini-2.0-flash"
        
        # Set up Groq API endpoint (instead of using the client library)
        self.groq_api_url = "https://api.groq.com/openai/v1/chat/completions"
        self.groq_model = "deepseek-r1-distill-llama-70b"
        
        # Conversation history
        self.conversation = []
        
    def gemini_agent(self, prompt):
        """Agent 1: Gemini model"""
        try:
            response = self.gemini_client.models.generate_content(
                model=self.gemini_model, 
                contents=prompt
            )
            return response.text
        except Exception as e:
            return f"Gemini Agent Error: {str(e)}"
    
    def groq_agent(self, prompt):
        """Agent 2: Groq model using direct API calls instead of the client library"""
        try:
            headers = {
                "Authorization": f"Bearer {self.groq_api_key}",
                "Content-Type": "application/json"
            }
            
            payload = {
                "model": self.groq_model,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.6,
                "max_tokens": 4096,
                "top_p": 0.95
            }
            
            response = requests.post(self.groq_api_url, headers=headers, json=payload)
            response_data = response.json()
            
            if 'choices' in response_data and len(response_data['choices']) > 0:
                return response_data['choices'][0]['message']['content']
            else:
                return f"Groq Agent Error: Unexpected response format - {response_data}"
                
        except Exception as e:
            return f"Groq Agent Error: {str(e)}"
    
    def determine_conversation_length(self, task):
        """Determine how many turns the agents should converse based on task complexity"""
        prompt = f"""
        You are tasked with determining the optimal number of conversation turns needed for two AI agents 
        to solve this task: "{task}"
        
        Consider:
        1. Task complexity (simple tasks need fewer turns)
        2. Need for brainstorming (creative tasks may need more turns)
        3. Need for refinement (tasks requiring precision may need more turns)
        
        Return only a number between 3 and 10 representing the optimal number of turns.
        """
        
        response = self.gemini_agent(prompt)
        try:
            turns = int(response.strip())
            return max(3, min(10, turns))  # Ensure between 3-10 turns
        except:
            return 5  # Default to 5 turns if parsing fails
    
    def generate_initial_prompts(self, task):
        """Generate initial specialized prompts for each agent"""
        gemini_prompt = f"""
        You are Agent 1 (Gemini 2.5 Pro), working collaboratively with Agent 2 (Deepseek Llama 70B) on this task:
        "{task}"
        
        Your specialties are:
        - Creative thinking and ideation
        - Structured planning
        - Considering multiple perspectives
        
        Begin by introducing yourself to Agent 2 and outline your initial thoughts on how to approach this task.
        Focus on the big picture and strategic elements.
        """
        
        groq_prompt = f"""
        You are Agent 2 (Deepseek Llama 70B), working collaboratively with Agent 1 (Gemini 2.5 Pro) on this task:
        "{task}"
        
        Your specialties are:
        - Detailed analysis and implementation
        - Technical precision
        - Pragmatic refinement of ideas
        
        Wait for Agent 1's initial thoughts, then respond with your specialized perspective.
        Feel free to build on their ideas while adding your technical insights.
        """
        
        return gemini_prompt, groq_prompt
    
    def generate_follow_up_prompts(self, conversation_history, turn_number, max_turns, task):
        """Generate follow-up prompts for continuing the conversation"""
        
        # Convert conversation history to a readable format for the models
        history_text = "\n\n".join([f"{'Agent 1 (Gemini)' if i%2==0 else 'Agent 2 (Deepseek)'}: {msg}" for i, msg in enumerate(conversation_history)])
        
        gemini_prompt = f"""
        You are Agent 1 (Gemini 2.5 Pro). Review the conversation so far about the task: "{task}"
        
        Conversation history:
        {history_text}
        
        This is turn {turn_number} of {max_turns}.
        
        {'As this is the final turn, work with Agent 2 to conclude and present a final solution or recommendation.' 
          if turn_number == max_turns else 
         'Continue the collaborative discussion, building on what has been shared so far.'}
        
        Be concise but insightful. Advance the solution forward meaningfully.
        """
        
        groq_prompt = f"""
        You are Agent 2 (Deepseek Llama 70B). Review the conversation so far about the task: "{task}"
        
        Conversation history:
        {history_text}
        
        This is turn {turn_number} of {max_turns}.
        
        {'As this is the final turn, work with Agent 1 to conclude and present a final solution or recommendation.' 
          if turn_number == max_turns else 
         'Continue the collaborative discussion, building on what has been shared so far.'}
        
        Be concise but insightful. Advance the solution forward meaningfully.
        """
        
        return gemini_prompt, groq_prompt
    
    def generate_summary(self, task, conversation):
        """Generate a summary of the collaboration and final output"""
        history_text = "\n\n".join([f"{'Agent 1 (Gemini)' if i%2==0 else 'Agent 2 (Deepseek)'}: {msg}" for i, msg in enumerate(conversation)])
        
        summary_prompt = f"""
        Review this conversation between two AI agents collaborating on the task: "{task}"
        
        Full conversation:
        {history_text}
        
        Provide:
        1. A concise summary of the key insights and ideas generated
        2. The final solution or approach recommended
        3. How the collaboration between agents enhanced the result
        
        Format your response as a structured final report.
        """
        
        return self.gemini_agent(summary_prompt)
    
    def collaborate(self, task):
        """Main function to run the collaborative process"""
        print(f"📋 Task: {task}\n")
        print("🔄 Determining optimal conversation length...")
        max_turns = self.determine_conversation_length(task)
        print(f"✅ Decided on {max_turns} turns of conversation\n")
        print("🤖 Beginning agent collaboration...\n")
        
        # Initial prompts
        gemini_prompt, groq_prompt = self.generate_initial_prompts(task)
        
        # First agent starts
        print("🔵 Agent 1 (Gemini 2.5 Pro) thinking...")
        gemini_response = self.gemini_agent(gemini_prompt)
        self.conversation.append(gemini_response)
        print(f"🔵 Agent 1 (Gemini): {gemini_response}\n")
        
        # Second agent responds
        print("🟠 Agent 2 (Deepseek Llama 70B) thinking...")
        groq_response = self.groq_agent(groq_prompt + "\n\nAgent 1 said: " + gemini_response)
        self.conversation.append(groq_response)
        print(f"🟠 Agent 2 (Deepseek): {groq_response}\n")
        
        # Continue the conversation for the determined number of turns
        for turn in range(2, max_turns + 1):
            print(f"--- Turn {turn}/{max_turns} ---\n")
            
            # Generate follow-up prompts
            gemini_prompt, groq_prompt = self.generate_follow_up_prompts(
                self.conversation, turn, max_turns, task
            )
            
            # Gemini agent's turn
            print("🔵 Agent 1 (Gemini 2.5 Pro) thinking...")
            gemini_response = self.gemini_agent(gemini_prompt)
            self.conversation.append(gemini_response)
            print(f"🔵 Agent 1 (Gemini): {gemini_response}\n")
            
            # Groq agent's turn
            print("🟠 Agent 2 (Deepseek Llama 70B) thinking...")
            groq_response = self.groq_agent(groq_prompt)
            self.conversation.append(groq_response)
            print(f"🟠 Agent 2 (Deepseek): {groq_response}\n")
        
        # Generate final summary
        print("🔄 Generating final summary and output...")
        summary = self.generate_summary(task, self.conversation)
        print("\n📊 FINAL OUTPUT:\n")
        print(summary)
        
        # Return results
        return {
            "task": task,
            "conversation_turns": max_turns,
            "conversation": self.conversation,
            "summary": summary
        }
    
    def save_results(self, results, filename=None):
        """Save the collaboration results to a file"""
        if filename is None:
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            filename = f"ai_collaboration_{timestamp}.json"
        
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"\n💾 Results saved to {filename}")

# Example usage
if __name__ == "__main__":
    # Set your API keys as environment variables or pass them directly
    try:
        gemini_key = os.environ.get("GEMINI_API_KEY")
        groq_key = os.environ.get("GROQ_API_KEY")
        
        if not gemini_key:
            gemini_key = input("Enter your Gemini API key: ")
        if not groq_key:
            groq_key = input("Enter your Groq API key: ")
            
        system = AIAgentSystem(gemini_api_key=gemini_key, groq_api_key=groq_key)
        
        # User task
        user_task = input("Enter task for AI agents to collaborate on: ")
        
        # Run collaboration
        results = system.collaborate(user_task)
        
        # Save results
        system.save_results(results)
    except Exception as e:
        print(f"Error: {str(e)}")

📋 Task: make a python square game

🔄 Determining optimal conversation length...
✅ Decided on 7 turns of conversation

🤖 Beginning agent collaboration...

🔵 Agent 1 (Gemini 2.5 Pro) thinking...
🔵 Agent 1 (Gemini): Hello Agent 2! I'm Agent 1, and I'm excited to collaborate with you on creating a Python square game.

My initial thoughts are to break down the project into manageable stages. Here's a preliminary outline of how we can approach this:

**Phase 1: Game Concept & Design**

*   **Determine the Game Type:** What kind of "square game" are we making? Is it a puzzle game like Tetris? A strategy game like Go or Checkers (played on a square board)? An arcade game involving squares as obstacles or projectiles? A simple game of claiming squares? We need to define the core gameplay. Let's brainstorm a few different options and weigh the pros and cons. My initial inclination is toward a more original concept, perhaps a puzzle or strategy game variant.
*   **Define Game Mechanics:** Once we

In [10]:
import os
import json
import time
import sys
import requests # Import requests for direct Groq API call
from google import genai
# Removed: from groq import Groq, RateLimitError, APIError

# --- Configuration ---
# Load API keys from environment variables or prompt user
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

if not GEMINI_API_KEY:
    GEMINI_API_KEY = input("Enter your Google AI Gemini API key: ")
if not GROQ_API_KEY:
    GROQ_API_KEY = input("Enter your Groq API key: ")

# Set API keys for the libraries
# No global config needed for genai.Client directly

# Model Configuration (Using models EXACTLY as requested and confirmed)
GEMINI_MODEL_NAME = "gemini-2.0-flash"
GROQ_MODEL_NAME = "deepseek-r1-distill-llama-70b"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

print(f"--- Using Specified Models ---")
print(f"Gemini: {GEMINI_MODEL_NAME}")
print(f"Groq: {GROQ_MODEL_NAME} (via direct requests)")
print(f"----------------------------")


# --- Agent Collaboration System ---

class EnhancedAIAgentCollaborator:
    def __init__(self, gemini_model=GEMINI_MODEL_NAME, groq_model=GROQ_MODEL_NAME, groq_api_key=GROQ_API_KEY, groq_api_url=GROQ_API_URL, gemini_api_key=GEMINI_API_KEY):
        """
        Initializes the collaborator system using genai.Client and direct requests for Groq
        with the specified models.
        """
        print("Initializing Enhanced AI Agent Collaborator...")

        # Model configuration
        self.gemini_model_name = gemini_model
        self.groq_model_name = groq_model
        self.groq_api_url = groq_api_url
        self.groq_api_key = groq_api_key
        self.gemini_api_key = gemini_api_key

        # --- Gemini Client Setup (using genai.Client as requested) ---
        try:
            self.gemini_client = genai.Client(api_key=self.gemini_api_key)
            # Consider adding a small test call if needed to confirm model access during init
            print(f"✅ Gemini client initialized for model: {self.gemini_model_name}")
        except Exception as e:
            print(f"❌ Error initializing Gemini client: {e}")
            raise

        # --- Groq Client Setup (using direct requests as requested) ---
        if not self.groq_api_key:
             raise ValueError("Groq API key is required for direct API calls.")
        print(f"✅ Groq setup configured for model: {self.groq_model_name} (via direct API call)")


        # Conversation state
        self.state = {
            "task": None,
            "conversation_history": [], # List of {"agent": name, "content": message} dicts
            "max_turns": 5, # Default, will be determined dynamically
            "current_turn": 0
        }
        print("Initialization complete.")

    def _call_gemini(self, prompt):
        """
        Sends a prompt to the Gemini model using client.models.generate_content.
        Adds basic retry logic.
        """
        print(f"🧠 Calling Gemini ({self.gemini_model_name})...")
        retries = 3
        delay = 5
        for attempt in range(retries):
            try:
                # Create content object with the prompt
                content = prompt

                # Call generate_content with correct parameters
                response = self.gemini_client.models.generate_content(
                    model=self.gemini_model_name,
                    contents=content,
                    generation_parameters={  # Use generation_parameters instead of generation_config
                        "temperature": 0.7,
                        "top_p": 0.9,
                        "top_k": 40,
                        "max_output_tokens": 8192
                    }
                )

                # Accessing the text content safely
                if hasattr(response, 'text'):
                    return response.text
                elif hasattr(response, 'parts') and response.parts:
                    return "".join(part.text for part in response.parts if hasattr(part, 'text'))
                elif hasattr(response, 'candidates') and response.candidates and hasattr(response.candidates[0], 'content') and hasattr(response.candidates[0].content, 'parts'):
                    return "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
                else:
                    print("⚠️ Gemini Warning: Received empty or blocked response.")
                    try:
                        print(f"Safety Feedback/Block Reason: {response.prompt_feedback}")
                    except Exception:
                        pass
                    return "[Gemini Error: Empty or Blocked Response]"

            except Exception as e:
                # Catch specific API errors if needed (e.g., InvalidArgumentError for model not found)
                print(f"⚠️ Gemini API Error (Attempt {attempt + 1}/{retries}) calling {self.gemini_model_name}: {e}")
                if attempt < retries - 1:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    delay *= 2
                else:
                    print(f"❌ Gemini API failed after {retries} attempts.")
                    return f"[Gemini Error: {str(e)}]"
        return "[Gemini Error: Max retries exceeded]"


    def _call_groq(self, messages):
        """
        Sends messages to the Groq model using direct HTTPS requests.
        Does NOT support streaming.
        """
        print(f"🧠 Calling Groq ({self.groq_model_name} via Direct API Call)...")
        full_response_content = ""
        retries = 3
        delay = 5

        headers = {
            "Authorization": f"Bearer {self.groq_api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": self.groq_model_name, # Use the specified model name
            "messages": messages,
            "temperature": 0.6,
            "max_tokens": 4096,
            "top_p": 0.95,
            "stream": False
        }

        for attempt in range(retries):
            try:
                response = requests.post(self.groq_api_url, headers=headers, json=payload, timeout=120) # Increased timeout for potentially slower models

                if response.status_code == 200:
                    response_data = response.json()
                    if 'choices' in response_data and len(response_data['choices']) > 0 and 'message' in response_data['choices'][0] and 'content' in response_data['choices'][0]['message']:
                        full_response_content = response_data['choices'][0]['message']['content']
                        print(f"🤖 Groq Agent (Response Received): {full_response_content[:150]}...")
                        return full_response_content.strip()
                    else:
                        # Handle cases like '{"error": "model_not_found"}' which might return 200 but have an error object
                        if 'error' in response_data:
                             print(f"❌ Groq API Error in Response (Model: {self.groq_model_name}): {response_data['error']}")
                             full_response_content = f"[Groq Error: {response_data['error']}]"
                        else:
                             print(f"⚠️ Groq Warning: Unexpected response format - {response_data}")
                             full_response_content = f"[Groq Error: Unexpected response format - {response_data}]"
                        break # Don't retry if format is wrong or error reported in JSON
                # Handle common HTTP errors
                elif response.status_code == 401:
                    print(f"❌ Groq API Error: 401 Unauthorized. Check your API key.")
                    return "[Groq Error: 401 Unauthorized]"
                elif response.status_code == 404: # Specifically check for 404 which often means model not found
                     print(f"❌ Groq API Error: 404 Not Found. Check if model '{self.groq_model_name}' is available via API endpoint '{self.groq_api_url}'.")
                     return f"[Groq Error: 404 Model Not Found - {self.groq_model_name}]"
                elif response.status_code == 429:
                    print(f"⚠️ Groq API Error: 429 Rate Limit Exceeded (Attempt {attempt + 1}/{retries}).")
                    if attempt < retries - 1:
                         retry_after = int(response.headers.get("retry-after", delay))
                         print(f"Retrying in {retry_after} seconds...")
                         time.sleep(retry_after)
                         delay += 2 # Linear backoff might be safer for rate limits
                         continue
                    else:
                         print(f"❌ Groq API failed due to rate limits after {retries} attempts.")
                         return "[Groq Error: Rate Limited]"
                else:
                     print(f"⚠️ Groq API Error (Attempt {attempt + 1}/{retries}): {response.status_code} - {response.text}")
                     full_response_content = f"[Groq Error: {response.status_code} - {response.text}]"
                     if attempt < retries - 1:
                          print(f"Retrying in {delay} seconds...")
                          time.sleep(delay)
                          delay *= 2
                          continue
                     else:
                          break

            except requests.exceptions.Timeout:
                 print(f"⚠️ Groq Request Error: Timeout after 120 seconds (Attempt {attempt + 1}/{retries}).")
                 if attempt < retries - 1:
                     print(f"Retrying in {delay} seconds...")
                     time.sleep(delay)
                     delay *= 2
                 else:
                     print(f"❌ Groq Request failed due to timeout after {retries} attempts.")
                     full_response_content = "[Groq Error: Request Timeout]"
            except requests.exceptions.RequestException as e:
                print(f"⚠️ Groq Request Error (Attempt {attempt + 1}/{retries}): {e}")
                if attempt < retries - 1:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    delay *= 2
                else:
                    print(f"❌ Groq Request failed after {retries} attempts.")
                    full_response_content = f"[Groq Error: RequestException - {str(e)}]"
            except Exception as e:
                print(f"⚠️ Groq General Error (Attempt {attempt + 1}/{retries}): {e}")
                if attempt < retries - 1:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    delay *= 2
                else:
                    print(f"❌ Groq failed after {retries} attempts.")
                    full_response_content = f"[Groq Error: {str(e)}]"

        print(f"❌ Groq call failed. Last status: {full_response_content}")
        return full_response_content


    def _get_history_string(self, max_messages=10):
        """Formats the recent conversation history for the prompts."""
        history_str = "\n---\n".join(
            [f"**{msg['agent']}**: {msg['content']}" for msg in self.state["conversation_history"][-max_messages:]]
        )
        return history_str if history_str else "No conversation history yet."

    def _add_to_history(self, agent_name, content):
        """Adds a message to the conversation history."""
        self.state["conversation_history"].append({"agent": agent_name, "content": content})

    def _determine_conversation_length(self, task):
        """
        Uses Gemini to estimate an appropriate number of turns for the task.
        """
        print("🤔 Determining optimal conversation length...")
        # Using the primary Gemini model for this task
        prompt = f"""
        Analyze the complexity of the following task and estimate the number of conversational turns
        (exchanges between two AI agents, {self.gemini_model_name} and {self.groq_model_name}) required for effective brainstorming, refinement, and solution development.

        Task: "{task}"

        Consider:
        - Task complexity (Simple: 3-4 turns, Moderate: 5-7 turns, Complex/Creative: 8-10 turns)
        - Need for deep brainstorming or multiple perspectives.
        - Need for iterative refinement and feedback.

        Provide ONLY a single integer between 3 and 10 representing the recommended number of turns.
        Example Output: 6
        """
        response = self._call_gemini(prompt)
        try:
            cleaned_response = response.strip().replace("`", "").replace("'", "").replace('"', '')
            turns = int(cleaned_response)
            validated_turns = max(3, min(10, turns))
            print(f"✅ Estimated conversation length: {validated_turns} turns.")
            return validated_turns
        except ValueError:
            print(f"⚠️ Could not parse estimated turns from Gemini response ('{response}'). Defaulting to 5.")
            return 5

    def _generate_agent_prompt(self, agent_type):
        """Generates the prompt for the specified agent type based on the current state."""
        task = self.state["task"]
        turn = self.state["current_turn"]
        max_turns = self.state["max_turns"]
        history = self._get_history_string()

        common_instructions = f"""
        **Current Task:** "{task}"
        **Collaboration Context:** You are working with another AI agent to accomplish this task.
        **Conversation Turn:** {turn} of {max_turns} (estimated).
        **Recent Conversation History:**
        {history}
        ---
        """
        if agent_type == "Gemini":
            role_prompt = f"""
            **Your Role (Agent 1 - Gemini - {self.gemini_model_name}): Strategist & Synthesizer**
            - Focus on high-level planning, creative ideation, and structuring the approach.
            - Synthesize information from the conversation and propose next steps or frameworks.
            - Ensure the collaboration stays aligned with the overall goal.

            **Your Task for this Turn:**
            {'Initiate the collaboration. Introduce yourself briefly, outline your initial understanding of the task, and propose a strategic direction or key areas to explore.' if turn == 1 else f'Review the latest message from Agent 2 (Groq - {self.groq_model_name}). Build upon their points, offer strategic refinements, synthesize the progress, or pose clarifying questions to guide the collaboration forward. Focus on the bigger picture. If nearing the end ({turn} >= {max_turns -1}), start guiding towards a consolidated solution.'}
            Be insightful and aim to advance the collaborative goal significantly. If you think a good solution is forming, mention it.
            """
            return common_instructions + role_prompt

        elif agent_type == "Groq":
            role_prompt = f"""
            **Your Role (Agent 2 - Groq - {self.groq_model_name}): Analyst & Refiner**
            - Focus on detailed analysis, technical feasibility, practical implementation details, and critical evaluation.
            - Refine ideas proposed by Agent 1, pointing out potential issues or areas for improvement.
            - Provide concrete examples or elaborate on specific aspects.

            **Your Task for this Turn:**
            {'Wait for Agent 1 (Gemini - {self.gemini_model_name}) to initiate. Once they provide input, respond by analyzing their proposal. Offer your detailed perspective, provide constructive critique, refine the ideas with practical considerations, or suggest specific implementation details. Focus on the specifics and feasibility.' if turn == 1 else f'Review the latest message from Agent 1 (Gemini - {self.gemini_model_name}). Analyze their strategic points or synthesis. Provide detailed feedback, constructive criticism, technical insights, or concrete refinements. Ensure the ideas are grounded and practical. If nearing the end ({turn} >= {max_turns -1}), focus on solidifying the details of the proposed solution.'}
            Be specific and analytical. Help ground the ideas and ensure feasibility. If you identify major flaws or alternative practical paths, highlight them.
            """
            return common_instructions + role_prompt
        else:
            raise ValueError("Unknown agent type")

    def _generate_critique_prompt(self):
        """Generates a prompt for Gemini to critique the final proposed solution."""
        task = self.state["task"]
        history = self._get_history_string(max_messages=20)

        prompt = f"""
        **Task:** You are an impartial evaluator using the {self.gemini_model_name} model.
        **Context:** Two AI agents (Gemini - Strategist - {self.gemini_model_name}, Groq - Analyst - {self.groq_model_name}) have collaborated on the following task: "{task}"
        **Full Conversation Summary:**
        {history}
        ---
        **Your Evaluation Task:**
        1. Review the entire conversation flow and the final proposed solution or ideas presented in the last few turns.
        2. Critically assess how well the collaboration addressed the original task prompt.
        3. Identify the main strengths of the proposed solution/outcome.
        4. Identify any weaknesses, gaps, unresolved issues, or areas needing further refinement.
        5. Provide a concise (2-3 paragraphs) evaluation focusing on the quality and completeness of the outcome relative to the task. Do not suggest new solutions, only evaluate what was produced.
        """
        return prompt

    def _generate_summary_prompt(self, critique):
        """Generates the final summary prompt, incorporating the critique."""
        task = self.state["task"]
        history = self._get_history_string(max_messages=30)

        prompt = f"""
        **Original Task:** "{task}"
        **Collaboration Process:** Two AI agents (Gemini - {self.gemini_model_name}, Groq - {self.groq_model_name}) collaborated over {self.state['max_turns']} turns using specific API methods.
        **Full Conversation Log:**
        {history}
        ---
        **Independent Critique of the Outcome:**
        {critique}
        ---
        **Your Final Task: Generate a Comprehensive Final Report**

        Produce a structured report using Markdown that includes:
        1.  **Executive Summary:** A brief overview of the task and the final outcome/solution.
        2.  **Collaboration Highlights:** Key insights, decisions, and breakthroughs achieved during the conversation. Mention how the distinct roles contributed.
        3.  **Detailed Final Solution/Proposal:** Clearly present the final recommended approach, plan, or answer developed through the collaboration. Be specific.
        4.  **Critique Integration:** Briefly acknowledge the strengths and weaknesses identified in the critique and how they relate to the final proposal. (Do not try to fix them now, just acknowledge).
        5.  **Conclusion:** A brief concluding statement about the collaborative effort.
        """
        return prompt

    def collaborate(self, task):
        """
        Main function to run the collaborative process between Gemini and Groq agents
        using the specified API interaction methods and models.
        """
        print(f"\n🚀 Starting Collaboration on Task: '{task}'")
        self.state = { # Reset state for new task
            "task": task,
            "conversation_history": [],
            "max_turns": self._determine_conversation_length(task),
            "current_turn": 0
        }

        agent1_name = f"Agent 1 (Gemini - {self.gemini_model_name})"
        agent2_name = f"Agent 2 (Groq - {self.groq_model_name})"

        # --- Collaboration Loop ---
        for i in range(self.state["max_turns"]):
            self.state["current_turn"] = i + 1
            print(f"\n--- Turn {self.state['current_turn']}/{self.state['max_turns']} ---")

            # Agent 1 (Gemini) Turn
            print(f"🔵 {agent1_name}'s turn...")
            gemini_prompt = self._generate_agent_prompt("Gemini")
            gemini_response = self._call_gemini(gemini_prompt)
            if "[Gemini Error:" in gemini_response:
                 print(f"Stopping collaboration due to Gemini error: {gemini_response}")
                 break
            self._add_to_history(agent1_name, gemini_response)
            print(f"🔵 {agent1_name}: {gemini_response}")

            # Agent 2 (Groq) Turn
            print(f"\n🟠 {agent2_name}'s turn...")
            groq_instruction_prompt = self._generate_agent_prompt("Groq")

            groq_messages_payload = []
            for msg in self.state["conversation_history"]:
                role = "user" if msg["agent"] == agent1_name else "assistant"
                groq_messages_payload.append({"role": role, "content": msg["content"]})
            groq_messages_payload.append({"role": "user", "content": groq_instruction_prompt})

            groq_response = self._call_groq(groq_messages_payload)
            if "[Groq Error:" in groq_response:
                 print(f"Stopping collaboration due to Groq error: {groq_response}")
                 break
            self._add_to_history(agent2_name, groq_response)

            time.sleep(1)

        print("\n--- Collaboration Phase Complete ---")

        # --- Critique Phase ---
        print("\n🔬 Generating Critique of the Collaboration...")
        critique_prompt = self._generate_critique_prompt()
        critique = self._call_gemini(critique_prompt)
        print(f"\nEvaluation Result:\n{critique}")

        # --- Summary Phase ---
        print("\n📑 Generating Final Summary Report...")
        summary_prompt = self._generate_summary_prompt(critique)
        summary = self._call_gemini(summary_prompt)
        print("\n📊 FINAL REPORT:\n")
        print(summary)

        # --- Return Results ---
        final_results = {
            "task": self.state["task"],
            "gemini_model_used": self.gemini_model_name,
            "groq_model_used": self.groq_model_name,
            "groq_interaction_method": "direct_requests",
            "estimated_turns": self.state["max_turns"],
            "collaboration_critique": critique,
            "final_report": summary,
            "full_conversation_history": self.state["conversation_history"],
        }
        return final_results

    def save_results(self, results, filename=None):
        """Saves the collaboration results to a JSON file."""
        if filename is None:
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            # Clean model names for filename
            safe_gemini_model = self.gemini_model_name.replace('/','_').replace(':','-')
            safe_groq_model = self.groq_model_name.replace('/','_').replace(':','-')
            filename = f"ai_collab_{safe_gemini_model}_{safe_groq_model}_{timestamp}.json"

        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=4, ensure_ascii=False)
            print(f"\n💾 Results successfully saved to {filename}")
        except Exception as e:
            print(f"❌ Error saving results to {filename}: {e}")


# --- Main Execution Block ---
if __name__ == "__main__":
    try:
        collaborator = EnhancedAIAgentCollaborator(
             gemini_model=GEMINI_MODEL_NAME,
             groq_model=GROQ_MODEL_NAME,
             groq_api_key=GROQ_API_KEY,
             groq_api_url=GROQ_API_URL,
             gemini_api_key=GEMINI_API_KEY
        )

        user_task = input("\n➡️ Enter the task for the AI agents to collaborate on: ")

        if user_task:
            results = collaborator.collaborate(user_task)
            save_choice = input("\n💾 Do you want to save the results to a JSON file? (yes/no): ").lower()
            if save_choice.startswith('y'):
                collaborator.save_results(results)
        else:
            print("No task entered. Exiting.")

    except ValueError as ve:
         print(f"\n❌ Configuration Error: {ve}")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred during the process: {e}")
        import traceback
        traceback.print_exc()

--- Using Specified Models ---
Gemini: gemini-2.0-flash
Groq: deepseek-r1-distill-llama-70b (via direct requests)
----------------------------
Initializing Enhanced AI Agent Collaborator...
✅ Gemini client initialized for model: gemini-2.0-flash
✅ Groq setup configured for model: deepseek-r1-distill-llama-70b (via direct API call)
Initialization complete.

🚀 Starting Collaboration on Task: 'make a new natural language'
🤔 Determining optimal conversation length...
🧠 Calling Gemini (gemini-2.0-flash)...
⚠️ Gemini API Error (Attempt 1/3) calling gemini-2.0-flash: Models.generate_content() got an unexpected keyword argument 'generation_parameters'
Retrying in 5 seconds...
⚠️ Gemini API Error (Attempt 2/3) calling gemini-2.0-flash: Models.generate_content() got an unexpected keyword argument 'generation_parameters'
Retrying in 10 seconds...
⚠️ Gemini API Error (Attempt 3/3) calling gemini-2.0-flash: Models.generate_content() got an unexpected keyword argument 'generation_parameters'
❌ Gemin

KeyboardInterrupt: 